In [1]:
import os
import numpy as np
import cv2
import random
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential

In [2]:
print(os.listdir('train'))

['without_mask', 'with_mask']


In [3]:
categories=['without_mask', 'with_mask']

In [4]:
categories.index(categories[0])

0

In [5]:
data=[]
for i in categories:
    path=os.path.join('train',i)
    
    label=categories.index(i)
    
    for file in os.listdir(path):
        image_path=os.path.join(path,file)
        img=cv2.imread(image_path)
        img=cv2.resize(img,(224,224))
        data.append([img,label])

In [6]:
len(data)

1376

In [7]:
## Now we are random shuffling
random.shuffle(data)

In [8]:
x=[]
y=[]

for features,label in data:
    x.append(features)
    y.append(label)

In [9]:
x=np.array(x)
y=np.array(y)

In [10]:
x.shape

(1376, 224, 224, 3)

In [11]:
y.shape

(1376,)

In [12]:
## Noramalization of features
x=x/255

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [14]:
x_train.shape

(1100, 224, 224, 3)

In [15]:
x_test.shape

(276, 224, 224, 3)

#### Now we have our data ,the only work remaining is to train it on Mobilenet V2

In [16]:
vgg=VGG16()

In [17]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [18]:
model = Sequential()

In [19]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [20]:
for layers in model.layers:
    layers.trainable=False

In [21]:
model.add(Dense(1,activation='sigmoid'))

In [22]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [23]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=5)

Epoch 1/5
35/35 [==============================] - 640s 14s/step - loss: 0.6274 - accuracy: 0.6209 - val_loss: 0.4417 - val_accuracy: 0.8768
Epoch 2/5
35/35 [==============================] - 442s 13s/step - loss: 0.3516 - accuracy: 0.9091 - val_loss: 0.3092 - val_accuracy: 0.8986
Epoch 3/5
35/35 [==============================] - 438s 13s/step - loss: 0.2583 - accuracy: 0.9418 - val_loss: 0.2651 - val_accuracy: 0.9094
Epoch 4/5
35/35 [==============================] - 440s 13s/step - loss: 0.2157 - accuracy: 0.9482 - val_loss: 0.2193 - val_accuracy: 0.9203
Epoch 5/5
35/35 [==============================] - 439s 13s/step - loss: 0.1895 - accuracy: 0.9473 - val_loss: 0.1965 - val_accuracy: 0.9384


In [54]:
## Now we are going to predict the values 
## For this we are making the function
def detect_mask(image):
    y_pred=model.predict_classes(image.reshape(1,224,224,3))
    return y_pred[0][0]

In [40]:
## Lets test with masked image

In [37]:
img=cv2.imread('test/with_mask/1-with-mask.jpg')
img=cv2.resize(img,(224,224))

In [39]:
detect_mask(img)[0][0]

1

In [53]:
## Code for face detection through web cam

In [64]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    img=cv2.resize(frame,(224,224))
    y_pred=detect_mask(img)
    if y_pred == 0:
        cv2.putText(frame,'No Mask',(30,30),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),2)
    if y_pred == 1:
        cv2.putText(frame,'Mask',(30,30),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),2)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detector=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    points=detector.detectMultiScale(gray)
    for (x, y, w, h) in points:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
    cv2.imshow('face',frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
cv2.destroyAllWindows()
cap.release()